# Ejemplo Simpy simulador de procesos

## Este ejemplo modela la caja de un banco y clientes que llegan T veces al azar. Cada cliente tiene una cierta paciencia. Cuando uno llega a caja se queda ahi un tiempo antes de irse.

In [1]:

import random
import simpy


RANDOM_SEED = 50  # Semilla randomica
NUEVOS_CLIENTES = 10  # Numero total de clientes
INTERVALO_CLIENTES = 10.0  # Genera nuevos clientes aproximadamente cada x segundos
PACIENCIA_MINIMA = 1  # Paciencia minima del cliente
PACIENCIA_MAXIMA = 3  # Paciencia maxima del cliente


def fuente(env, number, interval, counter):
    """Aqui se generan clientes al azar"""
    for i in range(number):
        c = cliente(env, 'Cliente%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def cliente(env, name, counter, time_in_bank):
    """Un cliente llega, es atendido y se va."""
    arrive = env.now
    print('%7.4f %s: Aqui estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(PACIENCIA_MINIMA, PACIENCIA_MAXIMA)
        # Espera llegar a la caja o si se acaba la paciencia se va
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # llega a la caja
            print('%7.4f %s: Esperó %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Terminó' % (env.now, name))

        else:
            # Se enoja y se va
            print('%7.4f %s: Se Enojó despues de %6.3f' % (env.now, name, wait))


# Iniciamos la simulación
print('Inicio de la simulación')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Iniciamos el proceso
counter = simpy.Resource(env, capacity=1)
env.process(fuente(env, NUEVOS_CLIENTES, INTERVALO_CLIENTES, counter))
env.run()

Inicio de la simulación
 0.0000 Cliente00: Aqui estoy
 0.0000 Cliente00: Esperó  0.000
 6.8823 Cliente01: Aqui estoy
 8.8287 Cliente01: Se Enojó despues de  1.946
 9.6594 Cliente02: Aqui estoy
10.8298 Cliente02: Se Enojó despues de  1.170
12.1738 Cliente00: Terminó
44.8376 Cliente03: Aqui estoy
44.8376 Cliente03: Esperó  0.000
45.9068 Cliente03: Terminó
48.6558 Cliente04: Aqui estoy
48.6558 Cliente04: Esperó  0.000
50.3201 Cliente05: Aqui estoy
51.6980 Cliente05: Se Enojó despues de  1.378
54.1670 Cliente06: Aqui estoy
55.8248 Cliente06: Se Enojó despues de  1.658
90.0574 Cliente07: Aqui estoy
92.2724 Cliente07: Se Enojó despues de  2.215
95.4349 Cliente04: Terminó
99.3154 Cliente08: Aqui estoy
99.3154 Cliente08: Esperó  0.000
109.3937 Cliente09: Aqui estoy
112.0435 Cliente09: Se Enojó despues de  2.650
138.7014 Cliente08: Terminó


### Fuente: Basado en el programa bank08.py del tutorial de Simpy 2 TheBank (KGM)